In [41]:
import ee
import geemap
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),os.pardir)))
from src import eetools as eet

In [42]:
try: 
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [43]:
coordenadas = "-48.53801472648439,-22.503806214013736,-48.270222978437516,-22.7281869567509"
x1, y1, x2, y2 = coordenadas.split(",")

datas = "2014-08-01,2014-12-01"
inicio, fim = datas.split(",")
escala = 30
dummy_value = 99999

geom = ee.Geometry.Polygon([[[float(x1),float(y2)],
                             [float(x2),float(y2)],
                             [float(x2),float(y1)],
                             [float(x1),float(y1)],
                             [float(x1),float(y2)]]])

latitude_central = (float(x1)+float(x2))/2
longitude_central = (float(y1)+float(y2))/2

collec = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(geom)\
    .filterDate(inicio,fim)\
    .filterMetadata('CLOUD_COVER','less_than',30)\
    .map(eet.ndvi)

In [171]:
ndvi_collec = ee.ImageCollection(collec\
    .select('ndvi')\
    .toList(999)\
    .map(
        lambda img: ee.Image(img)\
            .set(
                'system:index',
                ee.String(ee.Image(img).get('SENSING_TIME'))\
                    .replace('-|:|\\.','_','g')
            )
        ))\
    .sort('system:index')\
    .toBands()

ndvi_collec_bands = ndvi_collec\
    .bandNames()\
    .getInfo()

In [175]:
blended = ee.Image(dummy_value).blend(ndvi_collec)

In [176]:
df = eet.extrair_lonlat(blended, geom, ndvi_collec_bands, escala)
df

,2014_08_01T13_16_17_6581530Z_ndvi,2014_08_01T13_16_41_6000220Z_ndvi,2014_08_17T13_16_23_4096060Z_ndvi,2014_09_02T13_16_25_7726310Z_ndvi,2014_09_02T13_16_49_7102640Z_ndvi,2014_09_11T13_10_41_1118500Z_ndvi,2014_09_18T13_16_26_7334180Z_ndvi,2014_09_18T13_16_50_6752870Z_ndvi,2014_10_04T13_16_30_3092830Z_ndvi,2014_10_04T13_16_54_2553880Z_ndvi,2014_10_13T13_10_45_7041390Z_ndvi,2014_10_29T13_10_41_8501550Z_ndvi,2014_11_05T13_16_32_4161130Z_ndvi,latitude,longitude
0,99999.0,99999.0,99999.0,99999.0,99999.0,0.328715,99999.0,99999.0,99999.0,99999.0,0.482913,0.465691,99999.0,-22.728169,-48.537927
1,99999.0,99999.0,99999.0,99999.0,99999.0,0.705186,99999.0,99999.0,99999.0,99999.0,0.687115,0.750785,99999.0,-22.728169,-48.537627
2,99999.0,99999.0,99999.0,99999.0,99999.0,0.747272,99999.0,99999.0,99999.0,99999.0,0.724499,0.777710,99999.0,-22.728169,-48.537327
3,99999.0,99999.0,99999.0,99999.0,99999.0,0.709579,99999.0,99999.0,99999.0,99999.0,0.729393,0.819333,99999.0,-22.728169,-48.537027
4,99999.0,99999.0,99999.0,99999.0,99999.0,0.737570,99999.0,99999.0,99999.0,99999.0,0.742866,0.824793,99999.0,-22.728169,-48.536727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667211,99999.0,99999.0,99999.0,99999.0,99999.0,0.455296,99999.0,99999.0,99999.0,99999.0,0.238938,0.299978,99999.0,-22.503906,-48.271634
667212,99999.0,99999.0,99999.0,99999.0,99999.0,0.447883,99999.0,99999.0,99999.0,99999.0,0.242439,0.317909,99999.0,-22.503906,-48.271334
667213,99999.0,99999.0,99999.0,99999.0,99999.0,0.458235,99999.0,99999.0,99999.0,99999.0,0.240911,0.320276,99999.0,-22.503906,-48.271033
667214,99999.0,99999.0,99999.0,99999.0,99999.0,0.430154,99999.0,99999.0,99999.0,99999.0,0.241740,0.319300,99999.0,-22.503906,-48.270733
